In [1]:
## Run when initialise the code
import os

import geopandas as gpd
import osmnx as ox
from geopandas import GeoDataFrame, GeoSeries
from osmnx import io
import glob

project_crs = 'epsg:3857'
from sklearn.cluster import DBSCAN
from shapely.geometry import Polygon, Point, LineString, MultiPolygon, MultiPoint
import math
import warnings
import pandas as pd
import shutil
from tqdm import tqdm
import time

warnings.filterwarnings(action='ignore')
from momepy import remove_false_nodes, extend_lines

pjr_loc = os.path.dirname(os.getcwd())
import ast  # to convert str with list to list of string

In [3]:
## Choose locations and create folders if necessary

place = 'Turin'
print(place)
data_folder  = f'{pjr_loc}/places/{place.replace(",","_").replace(" ","_")}_test'
os.makedirs(f'{data_folder}/delete_2_nodes',exist_ok = True)
os.makedirs(f'{data_folder}/split_tp_intersection',exist_ok = True)
if place =='Tel Aviv':
    useful_tags_path = ['name:en','highway','length','bearing','tunnel','junction']
    ox.utils.config(useful_tags_way=useful_tags_path)
# Re
my_gdf = gpd.read_file(f'{data_folder}/osm_data.gpkg',layer='edges')
if place == 'Tel Aviv':
    my_gdf.rename(columns={'name:en': 'name'}, inplace=True)
my_gdf

Turin


In [ ]:
# Download data
graph = ox.graph_from_place(place, network_type='all')
graph = ox.bearing.add_edge_bearings(graph, precision=1)
graph_pro = ox.projection.project_graph(graph, to_crs=project_crs)
io.save_graph_geopackage(graph_pro, filepath=f'{data_folder}/osm_data.gpkg', encoding='utf-8', directed=False)

In [ ]:
## Filter out polylines and calculate angles
my_gdf = gpd.read_file(f'{data_folder}/osm_data.gpkg',
                       layer='edges')  # Identify roundabout elements, if any exist, and store them in a separate DataFrame.
if place =='Tel Aviv':
    my_gdf.rename(columns={'name:en':'name'}, inplace=True)

is_junction = True if 'junction' in my_gdf.columns else False
if is_junction:
    round_about = my_gdf[my_gdf['junction'].isin(['roundabout', 'circular'])]
    my_gdf = my_gdf[~((my_gdf['junction'] == 'roundabout') | (my_gdf['junction'] == 'circular'))]
to_remove = my_gdf[~((my_gdf['highway'] == 'motorway') | (my_gdf['highway'] == 'trunk')| (my_gdf['highway'] == 'motorway_link')| (my_gdf['highway'] == 'motorway_link')| (my_gdf['highway'] == 'trunk_link'))]
# Eliminate polylines that lack a name and calculate angles ranging from 0 to 180 degrees based on the bearing field.
df_pro = to_remove.to_crs(project_crs).dropna(subset=['name'])
df_pro = df_pro[df_pro['name']!='']
df_pro['angle'] = df_pro['bearing'].apply(lambda x: x if x < 180 else x - 180)
df_pro['length'] = df_pro.length